In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [ ]:
data=pd.read_csv("uber.csv")

In [ ]:
data

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data=data.drop(['Unnamed: 0'], axis=1)

In [ ]:
data=data.drop(['key'], axis=1)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
data['dropoff_longitude'].fillna(value=data['dropoff_longitude'].mean(), inplace=True)
data['dropoff_latitude'].fillna(value=data['dropoff_latitude'].median(), inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.dtypes

In [ ]:
data.pickup_datetime=pd.to_datetime(data.pickup_datetime, errors='coerce')

In [ ]:
data.dtypes

In [ ]:
data= data.assign(hour = data.pickup_datetime.dt.hour,
             day= data.pickup_datetime.dt.day,
             month = data.pickup_datetime.dt.month,
             year = data.pickup_datetime.dt.year,
             dayofweek = data.pickup_datetime.dt.dayofweek)

In [ ]:
data.head()

In [ ]:
data=data.drop(['pickup_datetime'], axis=1)

In [ ]:
data.head()

In [ ]:
data.plot(kind = "box",subplots = True,layout = (7,2),figsize=(15,20)) 

In [ ]:
#Using the InterQuartile Range to fill the values
def remove_outlier(data1 , col):
    Q1 = data1[col].quantile(0.25)
    Q3 = data1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1-1.5*IQR
    upper_whisker = Q3+1.5*IQR
    data[col] = np.clip(data1[col] , lower_whisker , upper_whisker)
    return data1

def treat_outliers_all(data1 , col_list):
    for c in col_list:
        data1 = remove_outlier(data , c)
    return data1

In [ ]:
data=treat_outliers_all(data,data.iloc[: , 0::])

In [ ]:
data.plot(kind = "box",subplots = True,layout = (7,2),figsize=(15,20))

In [ ]:
!pip install haversine 

In [ ]:
import haversine as hs

In [ ]:
travel_dist = []
for pos in range(len(data['pickup_longitude'])):
        long1,lati1,long2,lati2 = [data['pickup_longitude'][pos],data['pickup_latitude'][pos],data['dropoff_longitude'][pos],data['dropoff_latitude'][pos]]
        loc1=(lati1,long1)
        loc2=(lati2,long2)
        c = hs.haversine(loc1,loc2)
        travel_dist.append(c)

print(travel_dist)
data['dist_travel_km'] = travel_dist
data.head()

In [ ]:
x = data[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','hour','day','month','year','dayofweek','dist_travel_km']]
y = data['fare_amount']

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.33)

In [ ]:
from sklearn.linear_model import LinearRegression
regression = LinearRegression()

In [ ]:
regression.fit(X_train,y_train)

In [ ]:
regression.intercept_ #To find the linear intercept

In [ ]:
regression.coef_ #To find the linear coeeficient

In [ ]:
pred=regression.predict(X_test)

In [ ]:
pred

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,pred)

In [ ]:
MSE = mean_squared_error(y_test,pred)

In [ ]:
MSE

In [ ]:
RMSE = np.sqrt(MSE)

In [ ]:
RMSE

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=100) 

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
pred = rf.predict(X_test)

In [ ]:
pred

In [ ]:
R=r2_score(y_test,pred)

In [ ]:
MSE = mean_squared_error(y_test,pred)

In [ ]:
MSE

In [ ]:
RMSE = np.sqrt(MSE)

In [ ]:
RMSE